In [78]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import json

# Read recipe inputs
disk_usage = dataiku.Dataset("disk_usage")
df = disk_usage.get_dataframe()

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   status  1 non-null      object
 1   data    1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [80]:
df

,status,data
0,success,"{""result"":[{""metric"":{""cluster"":""mutu-100"",""ho..."


In [81]:
for key, value in df["data"].items():
    print(value)
    dict = json.loads(value)
dict

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'result': [{'metric': {'cluster': 'mutu-100',
    'hostname': 'broker-102',
    'job': 'node',
    'device': '/dev/sda1',
    'fstype': 'xfs',
    'dc': 'dalembert',
    'mountpoint': '/'},
   'values': [['1624601700', '16.318256082554285'],
    ['1624602600', '16.201162504168707'],
    ['1624603500', '16.240144417161986'],
    ['1624604400', '16.166954373278003'],
    ['1624605300', '16.24035425373792'],
    ['1624606200', '16.31835623182917'],
    ['1624607100', '16.201300805548314'],
    ['1624608000', '16.28062380026533'],
    ['1624608900', '16.407470010418862'],
    ['1624609800', '16.287958542396936'],
    ['1624610700', '16.336883847681733'],
    ['1624611600', '16.45413003448617'],
    ['1624612500', '16.4582743068609'],
    ['1624613400', '16.458279075874003'],
    ['1624614300', '16.380172179494778'],
    ['1624615200', '16.38534178968375'],
    ['1624616100', '16.326716311774987'],
    ['1624617000', '16.31181791488352'],
    ['1624617900', '16.466858530421945'],
    ['162

In [99]:
df = pd.json_normalize(dict["result"])
df = df.explode('values')
df[['Timestamp','CPU_Usage']] = pd.DataFrame(df['values'].tolist(),index=df.index)
df

,values,metric.cluster,metric.hostname,metric.job,metric.device,metric.fstype,metric.dc,metric.mountpoint,Timestamp,CPU_Usage
0,"[1624601700, 16.318256082554285]",mutu-100,broker-102,node,/dev/sda1,xfs,dalembert,/,1624601700,16.318256082554285
0,"[1624602600, 16.201162504168707]",mutu-100,broker-102,node,/dev/sda1,xfs,dalembert,/,1624602600,16.201162504168707
0,"[1624603500, 16.240144417161986]",mutu-100,broker-102,node,/dev/sda1,xfs,dalembert,/,1624603500,16.240144417161986
0,"[1624604400, 16.166954373278003]",mutu-100,broker-102,node,/dev/sda1,xfs,dalembert,/,1624604400,16.166954373278003
0,"[1624605300, 16.24035425373792]",mutu-100,broker-102,node,/dev/sda1,xfs,dalembert,/,1624605300,16.24035425373792
...,...,...,...,...,...,...,...,...,...,...
29,"[1628164800, 0.19235466536395052]",mutu-100,zook-105,node,/dev/sdb,xfs,diderot,/data,1628164800,0.19235466536395052
29,"[1628165700, 0.19236229848559105]",mutu-100,zook-105,node,/dev/sdb,xfs,diderot,/data,1628165700,0.19236229848559105
29,"[1628166600, 0.19236229848559105]",mutu-100,zook-105,node,/dev/sdb,xfs,diderot,/data,1628166600,0.19236229848559105
29,"[1628167500, 0.19236229848559105]",mutu-100,zook-105,node,/dev/sdb,xfs,diderot,/data,1628167500,0.19236229848559105


In [100]:
df.rename(columns={'metric.hostname': 'Broker', 'metric.dc': 'Data_Center','metric.mountpoint':'Disk'}, inplace=True)
df = df.drop(['metric.cluster', 'metric.job','metric.device','metric.fstype','values'], axis=1)
df

,Broker,Data_Center,Disk,Timestamp,CPU_Usage
0,broker-102,dalembert,/,1624601700,16.318256082554285
0,broker-102,dalembert,/,1624602600,16.201162504168707
0,broker-102,dalembert,/,1624603500,16.240144417161986
0,broker-102,dalembert,/,1624604400,16.166954373278003
0,broker-102,dalembert,/,1624605300,16.24035425373792
...,...,...,...,...,...
29,zook-105,diderot,/data,1628164800,0.19235466536395052
29,zook-105,diderot,/data,1628165700,0.19236229848559105
29,zook-105,diderot,/data,1628166600,0.19236229848559105
29,zook-105,diderot,/data,1628167500,0.19236229848559105


In [108]:
print(df['Broker'].unique())

['broker-102' 'broker-104' 'broker-106' 'broker-108' 'broker-110'
 'zook-102' 'zook-104' 'broker-101' 'broker-103' 'broker-105' 'broker-107'
 'broker-109' 'zook-101' 'zook-103' 'zook-105']


In [130]:
print(df.loc[,:])

        Broker Data_Center   Disk   Timestamp          CPU_Usage
7   broker-102   dalembert  /data  1624601700  85.19813931830231
7   broker-102   dalembert  /data  1624602600  85.19551916489941
7   broker-102   dalembert  /data  1624603500  85.36833767300556
7   broker-102   dalembert  /data  1624604400  85.50368443403465
7   broker-102   dalembert  /data  1624605300  85.65276416482965
..         ...         ...    ...         ...                ...
7   broker-102   dalembert  /data  1628164800   81.1835559268354
7   broker-102   dalembert  /data  1628165700  81.11972830105756
7   broker-102   dalembert  /data  1628166600  81.23866678467785
7   broker-102   dalembert  /data  1628167500  81.31334780568965
7   broker-102   dalembert  /data  1628168400  81.29351800339035

[3964 rows x 5 columns]


In [131]:
print(df.groupby(['Disk']))

In [105]:
dfx=pd.pivot_table(df,index=['Timestamp'],columns=df.groupby(['Timestamp']).cumcount().add(1),values=['CPU_Usage'],aggfunc='sum')
dfx

CPU_Usage                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
                            1                   2                   3                   4                   5                   6                   7                  8                  9                  10                 11                 12                  13                   14                  15                  16                  17                  18                 19                  20                  21                  22                 23                 24                 25                 26                 27                   28                   29                   30
Timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
1624601700  16.318256082554285  14.317783854878172   15.49957391252552  13.478442320145845  10.100068678557506   3.457606025056677  3.4971029914636063  85.19813931830231  74.49170188993246   70.9371542504953  73.32207046513899  88.85842354326587  0.2782959819247708  0.21633793356130582   15.75984280188294  15.526537912533342   10.51008934793714   13.46405420765494  9.834396497369724  3.4894725705204763  3.4707732701966876   3.176119796464249   81.1941608009878   76.2369075170215  73.17542698883462     72.76851249372  90.26527673586881   0.2519769785051267    0.236909196384957  0.24522166585246907
1624602600  16.201162504168707     14.317454792975  15.596985773890879  13.478451858172036  10.100073447570594     3.4575964870305  3.4987339939402062  85.19551916489941  74.54022059387908  70.97102483990011  73.39032168450653  88.95391639043964  0.2782959819247708  0.21633793356130582  15.798915326124913  15.604687730030363  10.588215320368704  13.542199256138872  9.893026744291575   3.469876695735863  3.4707828082228644   3.176224714752209  81.21675485531713  76.22728735315457  73.23356529134682  72.71504610569946  90.34340653349616   0.2519769785051267    0.236909196384957  0.24522166585246907
1624603500  16.240144417161986  14.200270603340726  15.479296068869132  13.478466165211302  10.021952244152104    3.45763940814831  3.4985336953904493  85.36833767300556  74.68795442810092  71.06013141969194  73.45062542738918  89.05680238136776  0.2782959819247708  0.21633793356130582  15.875767972061595  15.526571295624976  10.480893449803446  13.464078052720396  9.971639156058274  3.4893724212455908   3.470906802563192   3.156652685033052  81.28702746114351  76.35055053959594  73.21732270586183  72.74478912268337  90.58301465457083   0.2519769785051267    0.236909196384957  0.24522166585246907
1624604400  16.166954373278003  14.239343127582686  15.479367604065473  13.351567495913784  10.002427904563845  3.4588650445122937   3.498543233416626  85.50368443403465  74.62201677982304  71.02828475109524  73.53047764667299  89.18083478715981  0.2782959819247708  0.21633793356130582   15.87578227910086  15.487522616448445   10.51995166700614  13.464082821733484  9.971338708233645   3.4701342

In [0]:
# Write recipe outputs
json = dataiku.Dataset("Json")
json.write_with_schema(df)